In [1]:
import os
import cv2
import tensorflow as tf 
import pandas as pd
import numpy as np 
import keras as kr
import keras.layers as kl
from keras.optimizers import Adam, SGD
from sklearn.metrics import classification_report


In [3]:
# path dataset
train_path = "./dataset/AI Dataset/train"
test_path = "./dataset/AI Dataset/test"
valid_path = "./dataset/AI Dataset/valid"

In [8]:
# load dataset
def get_dataset(file_path):
    # load csv
    dataset = pd.read_csv(os.path.join(file_path, "_classes.csv"))

    # extract filename
    classes = dataset.columns
    file_names = dataset.iloc[:,0].values # dapetin file name dari kolom 0, .values ubah ke list
    # [baris, kolom]
    # mau dapetin labelnya sesuai dengan imagenya
    label = [] # nyimpen labelnya
    image = [] # nyimpen gambar
    max_label = dataset.iloc[:,1:].values # [[1, 0, 0, 0, 0, 0, 0], [] ... ]
    max_label = np.argmax(max_label, axis=1) # [0, 3, 1, 2]
    for i, img_path in enumerate(file_names):
        try:
            img = cv2.imread(os.path.join(file_path, img_path))
            img = img/255.0 # normalize
            if(max_label[i] != 6):
                # append
                label.append(max_label[i])
                image.append(img)
        except:
            continue
        
    # ubah menjadi data buat tensorflow, menjadi tuple
    tf_data = tf.data.Dataset.from_tensor_slices((image, label)).batch(16)   
    return tf_data, classes[1:-1]




In [ ]:
train_data, train_classes = get_dataset(train_path)
test_data, _ = get_dataset(test_path)
valid_data, _ = get_dataset(valid_path)

print(train_data, train_classes)

In [ ]:
# print(len(train_data))
print(train_data)

In [10]:
# alexnet model
def alexnet(num_of_classes):

    # convolution
    # filter: output kernel
    # kernel_size: kernel yang dipake
    # stride: jumlah stride(skip)

    # maxpool ga ada activation, cuman downsampling
    # di keras, padding itu ga usah dipeduliin, bilang aja same


    model = kr.Sequential([
        # width * height * channel
        kl.Input(shape=(224, 224, 3)),
        kl.Conv2D(filters=96, kernel_size=11, strides=4, activation="relu"),
        kl.MaxPool2D(pool_size=3, strides=2),
        kl.Conv2D(filters=256, kernel_size=5, padding="same", activation="relu"),
        kl.MaxPool2D(pool_size=3, strides=2),
        kl.Conv2D(filters=384, kernel_size=3, padding="same", activation="relu"),
        kl.Conv2D(filters=384, kernel_size=3, padding="same", activation="relu"),
        kl.Conv2D(filters=256, kernel_size=3, padding="same", activation="relu"),
        kl.MaxPool2D(pool_size=3, strides=2),

        # ubah jadi 1d, fully connected layer
        kl.Flatten(),
        kl.Dense(4096, activation="relu"),
        kl.Dropout(0.5),
        kl.Dense(4096, activation="relu"),
        kl.Dropout(0.5),
        # pake softmax agar memberikan probability score
        kl.Dense(num_of_classes, activation="softmax") # output
    ])

    return model

In [11]:
model = alexnet(len(train_classes))
# model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.compile(optimizer=SGD(learning_rate=0.001), loss="sparse_categorical_crossentropy", metrics=["accuracy"])


In [ ]:
model.fit(train_data, epochs=15, validation_data=valid_data)

In [ ]:
model.save("model.h5")

In [ ]:
# report
# test_img = []
# test_label = []

# for img_batch, label_batch in test_data:
#     # split
#     test_img.append(img_batch)
#     test_label.extend(label_batch)

# predicted = model.predict(test_img)

test_img = test_data.map(lambda x, y:x)
test_label = test_data.map(lambda x, y:y)
predicted = model.predict(test_img)

labels = []
for label in test_label:
    labels.extend(label)

In [ ]:
print(classification_report(labels, np.argmax(predicted, axis=1)))